# **`Project 2: Team Thomas Allinson`**

### **Objective**: Analyze the comparative costs of a vegan diet versus an omnivorous diet within the American population, with a specific focus on their environmental impact.

#### Group Members:
> Johann: johann.dicken@berkeley.edu <br>
> Laure: laureho@berkeley.edu <br>
> Reily: reilyjean@berkeley.edu <br>
> Carmen: carmenvega@berkeley.edu <br>
> Steven: k1519632@berkeley.edu <br>

### **[A]: Description of population of interest**

...descripition here...

### **[A]: Dietary Reference Intakes**

In [2]:
import pandas as pd
import numpy as np

In [3]:
# Import Dietary Requirements spreadsheet data as a pd.DataFrame
df = pd.read_csv('Dietary_Requirements.csv')
df.head()

,Nutrition,Source,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
0,Energy,---,1000.0,1200.0,1400.0,1600.0,1800.0,1800.0,2200.0,2000.0,2400.0,1800.0,2200.0,1600.0,2000.0
1,Protein,RDA,13.0,19.0,19.0,34.0,34.0,46.0,52.0,46.0,56.0,46.0,56.0,46.0,56.0
2,"Fiber, total dietary",---,14.0,16.8,19.6,22.4,25.2,25.2,30.8,28.0,33.6,25.2,30.8,22.4,28.0
3,"Folate, DFE",RDA,150.0,200.0,200.0,300.0,300.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0
4,"Calcium, Ca",RDA,700.0,1000.0,1000.0,1300.0,1300.0,1300.0,1300.0,1000.0,1000.0,1000.0,1000.0,1200.0,1000.0


Dietary function takes 2 arguments: `age`, a positive integer, and `sex`, a string (not case-senstitive) with the classification of male, female, or child.

In [4]:
def dietary_ref(age, sex):

    # Validate age input
    if not isinstance(age, int) or age <= 0:
        return "Incorrect age input. Please enter a positive integer for the age."
    
    # Normalize and validate sex input
    sex = sex.lower()
    if sex not in ['male', 'female', 'child']:
        return "Incorrect sex input. Input must be Male, Female, or Child."
    
    # Determine the appropriate column based on age and sex
    if sex == 'child':
        if age <= 3:
            col_name = 'C 1-3'
        elif age <= 8:
            col_name = 'C 4-8'
        else:
            return "Age out of range for child category."
    else:
        if age <= 8:
            col_name = f"{'F' if sex == 'female' else 'M'} 4-8"
        elif age <= 13:
            col_name = f"{'F' if sex == 'female' else 'M'} 9-13"
        elif age <= 18:
            col_name = f"{'F' if sex == 'female' else 'M'} 14-18"
        elif age <= 30:
            col_name = f"{'F' if sex == 'female' else 'M'} 19-30"
        elif age <= 50:
            col_name = f"{'F' if sex == 'female' else 'M'} 31-50"
        else:
            col_name = f"{'F' if sex == 'female' else 'M'} 51+"
    
    # Extract and return the relevant nutrient recommendations
    if col_name in df.columns:
        return df[['Nutrition', col_name]].set_index('Nutrition')[col_name]
    else:
        return "Matching column not found in DataFrame. Check the column names."

In [5]:
# Example usage
dietary_ref(15, 'Male')

Nutrition
Energy                            2200.0
Protein                             52.0
Fiber, total dietary                30.8
Folate, DFE                        400.0
Calcium, Ca                       1300.0
Carbohydrate, by difference        130.0
Iron, Fe                            11.0
Magnesium, Mg                      410.0
Niacin                              16.0
Phosphorus, P                     1250.0
Potassium, K                      4700.0
Riboflavin                           1.3
Thiamin                              1.2
Vitamin A, RAE                     900.0
Vitamin B-12                         2.4
Vitamin B-6                          1.3
Vitamin C, total ascorbic acid      75.0
Vitamin E (alpha-tocopherol)        15.0
Vitamin K (phylloquinone)           75.0
Zinc, Zn                            11.0
Name: M 14-18, dtype: float64

### **[A]: Data on prices for different foods**

Let's import our google spreadsheet as a pd.DataFrame here!

In [6]:
# prices_df = pd.read_csv('file_name.csv')
# prices_df

In [18]:

import re

# Load the CSV file into a DataFrame
df = pd.read_csv('min_cost_data_nutrients.csv')

# Define a regex pattern for common animal products
animal_product_pattern = r'\b(butter|cheese|milk|kefir|whey|eggnog|beef|chicken|pork|egg|fish|lamb|turkey|turtle|breast|mollusks|frog|thigh|yogurt|honey|gelatin|cream|lard|sausage|anchovy|shellfish|shrimp|mayo|ham|meat)\b'

# Create a new column 'animal product' that marks items based on the pattern
df['animal product'] = df['Ingredient description'].apply(
    lambda x: 'animal product' if re.search(animal_product_pattern, str(x), re.IGNORECASE) else 'plant-based'
)

# Display the updated DataFrame
df[df['animal product'] == 'plant-based'].tail(40)


,ingred_code,Ingredient description,Capric acid,Lauric acid,Myristic acid,Palmitic acid,Palmitoleic acid,Stearic acid,Oleic acid,Linoleic Acid,...,"Vitamin B-12, added",Vitamin B6,Vitamin C,Vitamin D,Vitamin E,"Vitamin E, added",Vitamin K,Water,Zinc,animal product
2673,43382,"Cranberry juice, unsweetened",0.000,0.000,0.000,0.008,0.001,0.002,0.022,0.042,...,0.0,0.052,9.3,0.0,1.20,0.00,5.1,87.13,0.10,plant-based
2674,43387,"Turnip greens, canned, no salt added",0.002,0.002,0.003,0.054,0.014,0.010,0.005,0.036,...,0.0,0.040,15.5,0.0,1.47,0.00,287.0,94.69,0.23,plant-based
2677,43392,"Hearts of palm, raw",0.000,0.001,0.001,0.034,0.000,0.004,0.005,0.076,...,0.0,0.810,8.0,0.0,0.50,0.00,0.0,69.50,3.73,plant-based
2678,43393,"Cereals ready-to-eat, POST, Shredded Wheat n' ...",0.000,0.000,0.004,0.405,0.014,0.021,0.316,1.166,...,0.0,0.080,0.0,0.0,1.09,0.00,2.0,3.14,3.13,plant-based
2681,43401,"Beverages, coffee, instant, with whitener, red...",0.000,0.000,0.000,7.368,0.000,17.761,1.705,0.576,...,0.0,0.000,0.0,0.0,0.26,0.00,3.8,6.16,0.02,plant-based
2682,43404,"Cranberry-apple juice drink, low calorie, with...",0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.019,32.0,0.0,0.27,0.00,1.1,95.20,0.02,plant-based
2684,43406,Yeast extract spread,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.000,0.0,0.0,0.00,0.00,0.0,40.90,4.19,plant-based
2685,43408,"Babyfood, juice, pear",0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.010,33.8,0.0,0.09,0.00,3.4,87.90,0.08,plant-based
2689,43441,"Rolls, pumpernickel",0.000,0.000,0.001,0.316,0.006,0.170,0.601,1.073,...,0.0,0.100,0.0,0.0,0.63,0.00,1.5,32.00,1.48,plant-based
2692,43479,"Alcoholic beverage, rice (sake)",0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.0,0.000,0.0,0.0,0.00,0.00,0.0,78.40,0.02,plant-based


### **[A]: Nutritional content of different foods**

...

### **[A]: Solution**

I think it'd be cool to make a graph for this :) For example, an overlying bar graph with different colors for sex, going across the x-axis with ages, y-axis being minimum diet cost.

In [29]:
# Code here

### **[B]: Is your solution edible?**

...

### **[B]: What is total cost for population of interest?**

In [31]:
# Import wbdata
# Code function for total cost

### **[C]: Sensitivity of Solution**

In [30]:
# Code here